In [1]:
import os
os.environ["NAPARI_ASYNC"] = "1"
#os.environ["NAPARI_PERFMON"] = "/Users/fukai/projects/napari-travali2/examples/prefmon.json"

In [2]:
import pandas as pd
import tensorstore as ts
from copy import deepcopy
spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": "/Volumes/Extreme SSD/aligned_image5_labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,0,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = '/Volumes/Extreme SSD/aligned_image5_image.zarr'
image = ts.open(image_spec).result()[:,0,:,:]
bboxes_df = pd.read_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df.csv').set_index(["frame","label"])
bboxes_df.head()

min_y  min_x  max_y  max_x
frame label                            
0     1         13   1865     57   1909
      2         19   4954     60   4994
      3         29   2832     70   2876
      4         35   1633     67   1669
      5         38   2756     80   2797

In [3]:
import numpy as np
import tensorstore as ts
import tensorstore_trackarr as tta
import json

print(labels.shape)
split_dict = {}
ta = tta.TrackArray(labels, split_dict, {}, bboxes_df)

(1965, 19991, 15437)


In [4]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()

In [5]:
widget = StateMachineWidget(viewer, ta, image, crop_size=2048)
viewer.window.add_dock_widget(widget, area="right")

/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (19991, 15437) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [   0 9297 6124]
INFO:napari_travali2._logging:Region selected: coords [   0 9297 6124]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(7249), np.int64(11345), None), slice(np.int64(4076), np.int64(8172), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(0.0), np.float64(9480.21992707859), np.float64(6270.068513429522))
DEBUG:napari_travali2._logging:data coordinates: [   0.         2231.21992708 2194.06851343]
INFO:napari_travali2._logging:clicked at [   0 2231 2194] at fra

In [6]:
widget._cropped_image_layer.translate

array([  nan, 9210., 5636.])

In [6]:
viewer.layers["Cropped Image"].data = image[:,1000:2000,1000:2000]
